In [ ]:
%load_ext autoreload
%autoreload 2
import transformers
import torch
import math
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from mcllm.model.llm import *
from tqdm import tqdm
import torch.utils.data as data
from mcllm.data.synthetic import LowRankDataset

In [ ]:
# # single prediction
# seq_len = 100
# batch_size = 1
# x = torch.Tensor(np.zeros((batch_size, seq_len)))

# llm = NanoBERT(dropout=0, n_embed=10)  # max_seq_len=seq_len)

# out = llm(x)
# print('shapes', x.shape, out.shape)